### Домашнее задание №3 

Присылайте выполненное задание аналогично предыдущим домашним работам

In [1]:
import math
import random
import networkx as nx
from pomegranate.distributions import Categorical
from pomegranate.hmm import DenseHMM

### Задание 1
Постройте случайный граф на $n=200$ вершинах, согласно модели Эрдёша—Реньи с вероятностью появления ребра $p=\frac{1}{2}$. Напишите свой код генерации случайного графа, не пользуясь встроенным в `networkx` генератором [`gnp_random_graph`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.gnp_random_graph.html) и подобными. Посчитайте кликовое число в среднем по 10-20 сгенерированных графах. Насколько кликовое число далёко от теоретического среднего в $2\log_{1/p}n$?

In [2]:
def er_graph(n, p):
    g = nx.Graph()
    g.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                g.add_edge(i, j)
    return g

def clique_number(g):
    return max((len(c) for c in nx.find_cliques(g)), default=0)

n = 200
p = 0.5
trials = 20

random.seed(42)
cliques = []
for _ in range(trials):
    g = er_graph(n, p)
    cliques.append(clique_number(g))

avg_clique = sum(cliques) / len(cliques)

print("clique sizes:", cliques)
print("average:", avg_clique)
print("theoretical:", 2 * math.log(n, 1 / p))

clique sizes: [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]
average: 11.0
theoretical: 15.287712379549449


В среднем получилось около 11, а формула даёт примерно 15.3.
Разница заметная при $n=200$: асимптотика сходится довольно медленно.


### Задание 2
Смоделируйте случайный граф в модели предпочтительного присоединения или другой отличной от Эрдёша—Реньи (согласно Вашему представлению о естественном росте, скажем, социальной сети). Опишите свою модель и приведите пример характеристик графа, которые отличаются в Вашей модели от соответствующих характеристик модели Эрдёша—Реньи.

In [3]:
def preferential_attachment(n, m0=4, m=2):
    g = nx.Graph()
    g.add_nodes_from(range(m0))
    for i in range(m0):
        for j in range(i + 1, m0):
            g.add_edge(i, j)
    targets = []
    for node in g.nodes():
        targets.extend([node] * g.degree(node))
    for new_node in range(m0, n):
        g.add_node(new_node)
        chosen = set()
        while len(chosen) < m:
            chosen.add(random.choice(targets))
        for v in chosen:
            g.add_edge(new_node, v)
        targets.extend(list(chosen))
        targets.extend([new_node] * g.degree(new_node))
    return g

n = 200
pa = preferential_attachment(n, m0=4, m=2)

avg_deg_pa = sum(dict(pa.degree()).values()) / n
avg_clust_pa = nx.average_clustering(pa)
max_deg_pa = max(dict(pa.degree()).values())

# Сравнение с ER-моделью с таким же средним числом рёбер
p_er = avg_deg_pa / (n - 1)
er = nx.gnp_random_graph(n, p_er, seed=2)
avg_deg_er = sum(dict(er.degree()).values()) / n
avg_clust_er = nx.average_clustering(er)
max_deg_er = max(dict(er.degree()).values())

print("PA avg degree:", avg_deg_pa)
print("PA clustering:", avg_clust_pa)
print("PA max degree:", max_deg_pa)
print("ER avg degree:", avg_deg_er)
print("ER clustering:", avg_clust_er)
print("ER max degree:", max_deg_er)


PA avg degree: 3.98
PA clustering: 0.09632137579196404
PA max degree: 28
ER avg degree: 4.07
ER clustering: 0.02275915750915751
ER max degree: 13


Модель: стартуем с маленькой клики, каждый новый узел присоединяется к $m$ существующим
с вероятностью, пропорциональной их степеням (предпочтительное присоединение).
Отличия от ER: разброс степеней и гораздо более крупные хабы,
а также более высокая кластеризация при той же средней степени вершины.

### Задание 3
Смоделируйте с помощью скрытой марковской модели следующую ситуацию. Гражданин Рассеянный в зависимости (почти) от погоды надевает либо футболку, либо плащ, либо тулуп. Когда жарко, он равновероятно надевает каждый из трёх видов одежды. Когда холодно, он не надевает плащ, а равновероятно надевает футболку либо тулуп. Когда температура средняя, он надевает исключительно тулуп либо плащ. Температура никогда не меняется резко (жаркая погода сменяется на холодную, и наоборот, — только через среднее состояние); при этом каждое состояние погоды равновероятно может остаться без изменений либо смениться на любое другое допустимое. На протяжении недели Рассеянный постил свои фото в Instagram в следующей одежде:
- пн: тулуп,
- вт: тулуп,
- ср: плащ,
- чт: футболка,
- пт: плащ,
- сб: плащ,
- вс: футболка.

Какая наиболее вероятная погода стояла на улице Бассейной в каждый из этих дней?

In [4]:
days = ["пн", "вт", "ср", "чт", "пт", "сб", "вс"]
obs = ["тулуп", "тулуп", "плащ", "футболка", "плащ", "плащ", "футболка"]

categ_map = {"тулуп": 0, "плащ": 1, "футболка": 2}
obs_idx = [categ_map[o] for o in obs]
obs_idxs = [[[o] for o in obs_idx]]

hot = Categorical(probs=[[1/3, 1/3, 1/3]])
mild = Categorical(probs=[[1/2, 1/2, 0.0]])
cold = Categorical(probs=[[1/2, 0.0, 1/2]])

edges = [
    [0.5, 0.5, 0.0],
    [1/3, 1/3, 1/3],
    [0.0, 0.5, 0.5],
]
starts = [1/3, 1/3, 1/3]

model = DenseHMM([hot, mild, cold], edges=edges, starts=starts)
path = model.viterbi(obs_idxs)[0].tolist()
states = ["жарко", "средне", "холодно"]
best_path = [states[i] for i in path]
print(dict(zip(days, best_path)))


{'пн': 'холодно', 'вт': 'холодно', 'ср': 'средне', 'чт': 'холодно', 'пт': 'средне', 'сб': 'средне', 'вс': 'холодно'}
